# 🏗️ YOLO 建築圖檢測系統 v5.0

**針對大物件優化 + 多階段檢測**

- ✅ 大物件偵測優化（馬桶、洗手台等）
- ✅ 小物件保持高精度
- ✅ 分階段檢測（大物件 + 小物件 → 合併結果）

---

## 1️⃣ 環境設定

In [1]:
# 安裝套件
!pip install -q ultralytics==8.3.0 pdf2image openpyxl opencv-python Pillow pandas albumentations pyyaml

# 檢查 GPU
!nvidia-smi

print("\n✅ 環境設定完成")

'pip' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC



✅ 環境設定完成


'nvidia-smi' ���O�����Υ~���R�O�B�i���檺�{���Χ妸�ɡC


In [2]:
# 匯入套件
import os
import cv2
import numpy as np
import pandas as pd
from pathlib import Path
from ultralytics import YOLO
from pdf2image import convert_from_path
from PIL import Image
import albumentations as A
import yaml
import shutil
from tqdm import tqdm

print("✅ 套件載入完成")

✅ 套件載入完成


## 2️⃣ 路徑與參數配置

In [ ]:
# ==================== 路徑配置 ====================
DATA_BASE = r"D:\DownloadD\Stanley\FS\Hengmei\program\henmei_datav5"

TRAIN_IMAGES = os.path.join(DATA_BASE, "train", "images")
TRAIN_LABELS = os.path.join(DATA_BASE, "train", "labels")
VAL_IMAGES = os.path.join(DATA_BASE, "valid", "images")
VAL_LABELS = os.path.join(DATA_BASE, "valid", "labels")
TEST_IMAGES = os.path.join(DATA_BASE, "test", "images")
TEST_LABELS = os.path.join(DATA_BASE, "test", "labels")

AUG_BASE = os.path.join(DATA_BASE, "augmented")
AUG_TRAIN_IMAGES = os.path.join(AUG_BASE, "train", "images")
AUG_TRAIN_LABELS = os.path.join(AUG_BASE, "train", "labels")
AUG_VAL_IMAGES = os.path.join(AUG_BASE, "valid", "images")
AUG_VAL_LABELS = os.path.join(AUG_BASE, "valid", "labels")
AUG_TEST_IMAGES = os.path.join(AUG_BASE, "test", "images")
AUG_TEST_LABELS = os.path.join(AUG_BASE, "test", "labels")

DATA_YAML = os.path.join(DATA_BASE, "data.yaml")

PDF_INPUT = r"D:\DownloadD\Stanley\FS\Hengmei\program\henmei_datav5\inputs\test_document.pdf"
MODEL_WEIGHTS = r"C:\Users\tnfsh\Downloads\runs\detect\train\weights\best.pt"
VIS_OUTPUT = r"D:\DownloadD\Stanley\FS\Hengmei\program\henmei_datav5\inference_vis"
EXCEL_OUTPUT = r"D:\DownloadD\Stanley\FS\Hengmei\program\henmei_datav5\detection_results.xlsx"

print("✅ 路徑配置完成")

✅ 路徑配置完成


In [5]:
# ==================== 訓練參數（針對大物件優化）====================

MODEL_SIZE = 'm'  # 使用最大模型
IMG_SIZE = 1024   # 大尺寸以涵蓋大小物件
EPOCHS = 150
BATCH_SIZE = 4
TARGET_TOTAL_IMAGES = 1000

# ⚠️ 關鍵：針對大物件調整
MOSAIC_STRENGTH = 0.8  # 提高 Mosaic（大物件受益）
CLOSE_MOSAIC = 50      # 最後 50 epochs 關閉 Mosaic

# ==================== 多階段推論參數 ====================

# PDF 轉換
INFERENCE_DPI = 600

# 階段 1: 大物件偵測
LARGE_CONF = 0.25      # 較高閾值（過濾假陽性）
LARGE_IOU = 0.5        # 標準 NMS
LARGE_MAX_DET = 100    # 大物件數量較少

# 階段 2: 小物件偵測  
SMALL_CONF = 0.08      # 低閾值（偵測小物件）
SMALL_IOU = 0.3        # 寬鬆 NMS（保留重疊）
SMALL_MAX_DET = 500    # 小物件數量多

# 物件尺寸定義（歸一化面積）
LARGE_OBJECT_THRESHOLD = 0.01  # 面積 > 1% 視為大物件

print(f"\n✅ 參數設定完成")
print(f"   模型: YOLOv8{MODEL_SIZE}")
print(f"   圖像尺寸: {IMG_SIZE}")
print(f"   Mosaic: {MOSAIC_STRENGTH}")
print(f"\n多階段推論:")
print(f"   大物件 - 信心:{LARGE_CONF} IOU:{LARGE_IOU}")
print(f"   小物件 - 信心:{SMALL_CONF} IOU:{SMALL_IOU}")


✅ 參數設定完成
   模型: YOLOv8m
   圖像尺寸: 1024
   Mosaic: 0.8

多階段推論:
   大物件 - 信心:0.25 IOU:0.5
   小物件 - 信心:0.08 IOU:0.3


## 3️⃣ 資料增強（大物件優化）

In [6]:
def create_augmentation_pipeline(img_size):
    """
    針對大物件優化的增強管線
    
    關鍵：
    - 增加縮放變化（大物件在不同圖紙中尺寸差異大）
    - 保持完整物件（避免過度裁切）
    - 加強對比（大物件輪廓更明顯）
    """
    
    return A.Compose([
        # 幾何變換
        A.HorizontalFlip(p=0.5),
        A.VerticalFlip(p=0.3),
        A.Rotate(limit=10, p=0.4),  # 增加旋轉
        
        # ⚠️ 關鍵：大物件需要更多縮放變化
        A.Affine(
            scale=(0.7, 1.3),  # 更大的縮放範圍
            translate_percent=(-0.1, 0.1),
            p=0.5
        ),
        
        # 適度裁切（不要切掉大物件）
        A.RandomSizedBBoxSafeCrop(
            height=int(img_size * 0.85),
            width=int(img_size * 0.85),
            erosion_rate=0.2,
            p=0.4
        ),
        
        # 顏色增強（強化對比）
        A.CLAHE(clip_limit=3.0, p=0.6),  # 提高對比
        A.RandomBrightnessContrast(
            brightness_limit=0.2,
            contrast_limit=0.3,  # 加強對比
            p=0.6
        ),
        
        # 銳化（大物件邊緣）
        A.Sharpen(alpha=(0.2, 0.5), lightness=(0.8, 1.2), p=0.5),
        
        # 輕微模糊
        A.OneOf([
            A.MotionBlur(blur_limit=3, p=1),
            A.GaussianBlur(blur_limit=3, p=1),
        ], p=0.2),
        
    ], bbox_params=A.BboxParams(
        format='yolo',
        label_fields=['class_labels'],
        min_area=5,
        min_visibility=0.3  # 大物件可接受更多遮擋
    ))

print("✅ 增強管線建立完成")

✅ 增強管線建立完成


In [7]:
def augment_dataset(source_img_dir, source_label_dir, 
                   target_img_dir, target_label_dir,
                   target_count, img_size, set_name):
    """資料增強函數"""
    
    Path(target_img_dir).mkdir(parents=True, exist_ok=True)
    Path(target_label_dir).mkdir(parents=True, exist_ok=True)
    
    img_files = [f for f in os.listdir(source_img_dir) 
                 if f.lower().endswith(('.jpg', '.jpeg', '.png'))]
    
    if not img_files:
        print(f"⚠️ {set_name}: 沒有找到圖像")
        return
    
    print(f"\n{set_name}: 原始 {len(img_files)} 張 → 目標 {target_count} 張")
    
    # 複製原始圖像
    copied_count = 0
    valid_files = []
    
    for img_file in img_files:
        label_file = img_file.rsplit('.', 1)[0] + '.txt'
        label_path = os.path.join(source_label_dir, label_file)
        
        if not os.path.exists(label_path) or os.path.getsize(label_path) == 0:
            continue
        
        valid_files.append(img_file)
        
        shutil.copy2(
            os.path.join(source_img_dir, img_file),
            os.path.join(target_img_dir, img_file)
        )
        shutil.copy2(
            label_path,
            os.path.join(target_label_dir, label_file)
        )
        copied_count += 1
    
    print(f"  ✓ 複製 {copied_count} 張原始圖像")
    
    if not valid_files:
        return
    
    need_aug = target_count - copied_count
    if need_aug <= 0:
        return
    
    aug_pipeline = create_augmentation_pipeline(img_size)
    augmented_count = 0
    
    with tqdm(total=need_aug, desc=f"  增強 {set_name}") as pbar:
        while augmented_count < need_aug:
            img_file = np.random.choice(valid_files)
            base_name = img_file.rsplit('.', 1)[0]
            label_file = base_name + '.txt'
            
            img_path = os.path.join(source_img_dir, img_file)
            label_path = os.path.join(source_label_dir, label_file)
            
            img = cv2.imread(img_path)
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            
            with open(label_path, 'r') as f:
                lines = f.readlines()
            
            bboxes = []
            class_labels = []
            
            for line in lines:
                parts = line.strip().split()
                if len(parts) >= 5:
                    class_id = int(parts[0])
                    x_center, y_center, width, height = map(float, parts[1:5])
                    bboxes.append([x_center, y_center, width, height])
                    class_labels.append(class_id)
            
            if not bboxes:
                continue
            
            try:
                augmented = aug_pipeline(
                    image=img,
                    bboxes=bboxes,
                    class_labels=class_labels
                )
                
                if not augmented['bboxes']:
                    continue
                
                aug_img = cv2.cvtColor(augmented['image'], cv2.COLOR_RGB2BGR)
                aug_img_name = f"{base_name}_aug{augmented_count}.jpg"
                cv2.imwrite(
                    os.path.join(target_img_dir, aug_img_name),
                    aug_img
                )
                
                aug_label_name = f"{base_name}_aug{augmented_count}.txt"
                with open(os.path.join(target_label_dir, aug_label_name), 'w') as f:
                    for bbox, cls in zip(augmented['bboxes'], augmented['class_labels']):
                        f.write(f"{cls} {bbox[0]} {bbox[1]} {bbox[2]} {bbox[3]}\n")
                
                augmented_count += 1
                pbar.update(1)
                
            except:
                continue
    
    print(f"  ✓ 完成: {copied_count + augmented_count} 張")

print("✅ 增強函數定義完成")

✅ 增強函數定義完成


In [8]:
# 執行資料增強
print("="*60)
print("開始資料增強")
print("="*60)

train_target = int(TARGET_TOTAL_IMAGES * 0.7)
val_target = int(TARGET_TOTAL_IMAGES * 0.2)
test_target = int(TARGET_TOTAL_IMAGES * 0.1)

augment_dataset(TRAIN_IMAGES, TRAIN_LABELS, AUG_TRAIN_IMAGES, AUG_TRAIN_LABELS, train_target, IMG_SIZE, "訓練集")
augment_dataset(VAL_IMAGES, VAL_LABELS, AUG_VAL_IMAGES, AUG_VAL_LABELS, val_target, IMG_SIZE, "驗證集")
augment_dataset(TEST_IMAGES, TEST_LABELS, AUG_TEST_IMAGES, AUG_TEST_LABELS, test_target, IMG_SIZE, "測試集")

print("\n✅ 資料增強完成")

開始資料增強

訓練集: 原始 366 張 → 目標 700 張
  ✓ 複製 366 張原始圖像


  增強 訓練集: 100%|██████████| 334/334 [00:07<00:00, 44.89it/s]


  ✓ 完成: 700 張

驗證集: 原始 35 張 → 目標 200 張
  ✓ 複製 35 張原始圖像


  增強 驗證集: 100%|██████████| 165/165 [00:03<00:00, 47.88it/s]


  ✓ 完成: 200 張

測試集: 原始 18 張 → 目標 100 張
  ✓ 複製 18 張原始圖像


  增強 測試集: 100%|██████████| 82/82 [00:01<00:00, 48.22it/s]

  ✓ 完成: 100 張

✅ 資料增強完成


## 4️⃣ 更新 data.yaml

In [9]:
yaml_content = f"""
path: {AUG_BASE}
train: train/images
val: valid/images
test: test/images

# ⚠️ 請根據你的類別修改
names:
  0: AJ2
  1: DL2a
  2: maton-1
  3: maton-2
  4: PL-T-1
  5: sink-1
"""

aug_yaml_path = os.path.join(AUG_BASE, "data.yaml")
with open(aug_yaml_path, 'w', encoding='utf-8') as f:
    f.write(yaml_content)

print(f"✅ data.yaml 已更新")
print("\n⚠️ 請修改上方 names 欄位")

✅ data.yaml 已更新

⚠️ 請修改上方 names 欄位


## 5️⃣ 模型訓練（大物件優化）

In [10]:
print("="*60)
print(f"開始訓練 YOLOv8{MODEL_SIZE}")
print("="*60)

model = YOLO(f'yolov8{MODEL_SIZE}.pt')

results = model.train(
    data=aug_yaml_path,
    epochs=EPOCHS,
    imgsz=IMG_SIZE,
    batch=BATCH_SIZE,
    
    # ⚠️ 針對大物件優化
    mosaic=MOSAIC_STRENGTH,      # 提高 Mosaic（大物件受益）
    close_mosaic=CLOSE_MOSAIC,   # 最後關閉 Mosaic
    mixup=0.1,                   # 加入 Mixup
    copy_paste=0.1,              # 複製貼上增強
    
    # 損失權重（平衡大小物件）
    box=7.5,
    cls=0.5,
    dfl=1.5,
    
    # 優化器
    optimizer='AdamW',
    lr0=0.001,
    lrf=0.01,
    momentum=0.937,
    weight_decay=0.0005,
    
    # 其他
    patience=50,
    save=True,
    save_period=20,
    device=0,
    workers=8,
    project='runs/detect',
    name='train',
    exist_ok=True,
    val=True,
    plots=True,
)

print("\n✅ 訓練完成")
print(f"最佳模型: runs/detect/train/weights/best.pt")

開始訓練 YOLOv8m
New https://pypi.org/project/ultralytics/8.3.225 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.223  Python-3.10.18 torch-2.5.1 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=50, cls=0.5, compile=False, conf=None, copy_paste=0.1, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=D:\DownloadD\Stanley\FS\Hengmei\program\henmei_datav5\augmented\data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=150, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=1024, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.001, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.1, mode=train, model=yolov8m.pt, momentum=0.937,

## 6️⃣ 多階段推論（分別檢測大小物件）

In [11]:
def preprocess_image(image_np):
    """圖像前處理"""
    img = image_np.copy()
    
    # 雙邊濾波
    img = cv2.bilateralFilter(img, d=5, sigmaColor=50, sigmaSpace=50)
    
    # CLAHE
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    for i in range(3):
        img[:, :, i] = clahe.apply(img[:, :, i])
    
    # 銳化
    kernel = np.array([[-1, -1, -1], [-1, 9, -1], [-1, -1, -1]])
    img = cv2.filter2D(img, -1, kernel)
    
    return img

def calculate_iou(box1, box2):
    """計算 IOU"""
    x1_min, y1_min, x1_max, y1_max = box1
    x2_min, y2_min, x2_max, y2_max = box2
    
    inter_x_min = max(x1_min, x2_min)
    inter_y_min = max(y1_min, y2_min)
    inter_x_max = min(x1_max, x2_max)
    inter_y_max = min(y1_max, y2_max)
    
    if inter_x_max < inter_x_min or inter_y_max < inter_y_min:
        return 0.0
    
    inter_area = (inter_x_max - inter_x_min) * (inter_y_max - inter_y_min)
    box1_area = (x1_max - x1_min) * (y1_max - y1_min)
    box2_area = (x2_max - x2_min) * (y2_max - y2_min)
    union_area = box1_area + box2_area - inter_area
    
    return inter_area / union_area if union_area > 0 else 0.0

def merge_detections(large_dets, small_dets, iou_threshold=0.3):
    """
    合併大小物件檢測結果
    
    規則：
    - 大物件優先（信心度高）
    - 小物件去除與大物件重疊過多的
    """
    merged = large_dets.copy()
    
    for small_det in small_dets:
        small_box = [small_det['x1'], small_det['y1'], small_det['x2'], small_det['y2']]
        
        # 檢查是否與大物件重疊
        is_overlapping = False
        for large_det in large_dets:
            large_box = [large_det['x1'], large_det['y1'], large_det['x2'], large_det['y2']]
            iou = calculate_iou(small_box, large_box)
            
            if iou > iou_threshold:
                is_overlapping = True
                break
        
        # 不重疊才加入
        if not is_overlapping:
            merged.append(small_det)
    
    return merged

print("✅ 工具函數定義完成")

✅ 工具函數定義完成


In [12]:
def run_multi_stage_inference(pdf_path, model_path, output_folder=None):
    """
    多階段推論主函數
    
    階段 1: 偵測大物件（高閾值、標準 NMS）
    階段 2: 偵測小物件（低閾值、寬鬆 NMS）
    階段 3: 合併結果（去除重疊）
    """
    print("\n" + "="*60)
    print("多階段推論")
    print("="*60)
    print(f"DPI: {INFERENCE_DPI}")
    print(f"階段 1 (大物件): 信心 {LARGE_CONF}, IOU {LARGE_IOU}")
    print(f"階段 2 (小物件): 信心 {SMALL_CONF}, IOU {SMALL_IOU}")
    print("="*60)
    
    # 載入模型
    model = YOLO(model_path)
    class_names = model.names
    
    # 轉換 PDF
    print(f"\n轉換 PDF (DPI={INFERENCE_DPI})...")
    images = convert_from_path(pdf_path, dpi=INFERENCE_DPI)
    print(f"✓ 轉換完成: {len(images)} 頁")
    
    # 初始化結果
    all_detections = []
    page_counts = {}
    stage_stats = {'large': 0, 'small': 0, 'merged': 0}
    
    # 逐頁推論
    for page_idx, pil_img in enumerate(images, 1):
        print(f"\n{'='*40}")
        print(f"處理第 {page_idx}/{len(images)} 頁")
        print(f"{'='*40}")
        
        img_np = np.array(pil_img)
        img_h, img_w = img_np.shape[:2]
        img_area = img_h * img_w
        
        # 前處理
        img_processed = preprocess_image(img_np)
        
        # ============ 階段 1: 大物件偵測 ============
        print("\n階段 1: 偵測大物件...")
        results_large = model.predict(
            img_processed,
            conf=LARGE_CONF,
            iou=LARGE_IOU,
            max_det=LARGE_MAX_DET,
            imgsz=IMG_SIZE,
            verbose=False
        )
        
        large_detections = []
        if results_large and len(results_large) > 0:
            boxes = results_large[0].boxes
            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    w, h = x2 - x1, y2 - y1
                    area_ratio = (w * h) / img_area
                    
                    # 只保留大物件
                    if area_ratio >= LARGE_OBJECT_THRESHOLD:
                        conf = float(box.conf[0].cpu().numpy())
                        cls = int(box.cls[0].cpu().numpy())
                        
                        large_detections.append({
                            'page': page_idx,
                            'class': class_names[cls],
                            'confidence': conf,
                            'x1': int(x1), 'y1': int(y1),
                            'x2': int(x2), 'y2': int(y2),
                            'width': int(w), 'height': int(h),
                            'stage': 'large'
                        })
        
        print(f"  ✓ 偵測到 {len(large_detections)} 個大物件")
        for det in large_detections:
            print(f"    - {det['class']}: {det['confidence']:.2f}")
        
        # ============ 階段 2: 小物件偵測 ============
        print("\n階段 2: 偵測小物件...")
        results_small = model.predict(
            img_processed,
            conf=SMALL_CONF,
            iou=SMALL_IOU,
            max_det=SMALL_MAX_DET,
            imgsz=IMG_SIZE,
            verbose=False
        )
        
        small_detections = []
        if results_small and len(results_small) > 0:
            boxes = results_small[0].boxes
            if boxes is not None:
                for box in boxes:
                    x1, y1, x2, y2 = box.xyxy[0].cpu().numpy()
                    w, h = x2 - x1, y2 - y1
                    area_ratio = (w * h) / img_area
                    
                    # 只保留小物件
                    if area_ratio < LARGE_OBJECT_THRESHOLD:
                        conf = float(box.conf[0].cpu().numpy())
                        cls = int(box.cls[0].cpu().numpy())
                        
                        small_detections.append({
                            'page': page_idx,
                            'class': class_names[cls],
                            'confidence': conf,
                            'x1': int(x1), 'y1': int(y1),
                            'x2': int(x2), 'y2': int(y2),
                            'width': int(w), 'height': int(h),
                            'stage': 'small'
                        })
        
        print(f"  ✓ 偵測到 {len(small_detections)} 個小物件")
        
        # ============ 階段 3: 合併結果 ============
        print("\n階段 3: 合併結果...")
        page_detections = merge_detections(large_detections, small_detections)
        
        print(f"  ✓ 合併後總共 {len(page_detections)} 個物件")
        print(f"    大物件: {len(large_detections)}")
        print(f"    小物件: {len(page_detections) - len(large_detections)}")
        
        all_detections.extend(page_detections)
        stage_stats['large'] += len(large_detections)
        stage_stats['small'] += len(small_detections)
        stage_stats['merged'] += len(page_detections)
        
        # 統計
        page_counts[page_idx] = {}
        for det in page_detections:
            cls_name = det['class']
            page_counts[page_idx][cls_name] = page_counts[page_idx].get(cls_name, 0) + 1
    
    # 總結
    print("\n" + "="*60)
    print("推論完成")
    print("="*60)
    print(f"階段統計:")
    print(f"  大物件偵測: {stage_stats['large']}")
    print(f"  小物件偵測: {stage_stats['small']}")
    print(f"  合併後總數: {stage_stats['merged']}")
    print(f"  去除重複: {stage_stats['large'] + stage_stats['small'] - stage_stats['merged']}")
    
    # 類別統計
    total_counts = {}
    for det in all_detections:
        cls = det['class']
        total_counts[cls] = total_counts.get(cls, 0) + 1
    
    print("\n各類別總數:")
    for cls, count in sorted(total_counts.items()):
        print(f"  {cls}: {count}")
    
    return total_counts, page_counts, all_detections

print("✅ 推論函數定義完成")

✅ 推論函數定義完成


In [13]:
# 執行多階段推論
total_counts, page_counts, all_detections = run_multi_stage_inference(
    pdf_path=PDF_INPUT,
    model_path=MODEL_WEIGHTS,
    output_folder=VIS_OUTPUT
)


多階段推論
DPI: 600
階段 1 (大物件): 信心 0.25, IOU 0.5
階段 2 (小物件): 信心 0.08, IOU 0.3


FileNotFoundError: [Errno 2] No such file or directory: 'D:\\DownloadD\\Stanley\\FS\\Hengmei\\program\\henmei_datav5\\runs\\train\\weights\\best.pt'

## 7️⃣ 視覺化（標示大小物件）

In [ ]:
def visualize_multi_stage_detections(pdf_path, all_detections, output_folder, dpi):
    """
    視覺化多階段檢測結果
    
    - 大物件用粗框 + 紅色
    - 小物件用細框 + 綠色
    """
    print(f"\n生成視覺化...")
    Path(output_folder).mkdir(parents=True, exist_ok=True)
    
    images = convert_from_path(pdf_path, dpi=dpi)
    
    for page_idx, pil_img in enumerate(images, 1):
        img_np = np.array(pil_img)
        img_bgr = cv2.cvtColor(img_np, cv2.COLOR_RGB2BGR)
        
        page_dets = [d for d in all_detections if d['page'] == page_idx]
        
        for det in page_dets:
            x1, y1, x2, y2 = det['x1'], det['y1'], det['x2'], det['y2']
            cls_name = det['class']
            conf = det['confidence']
            stage = det['stage']
            
            # 根據階段選擇顏色和線寬
            if stage == 'large':
                color = (0, 0, 255)  # 紅色
                thickness = 4
                label = f"[大] {cls_name} {conf:.2f}"
            else:
                color = (0, 255, 0)  # 綠色
                thickness = 2
                label = f"[小] {cls_name} {conf:.2f}"
            
            # 繪製
            cv2.rectangle(img_bgr, (x1, y1), (x2, y2), color, thickness)
            
            (label_w, label_h), _ = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.6, 2)
            cv2.rectangle(img_bgr, (x1, y1 - label_h - 10), (x1 + label_w, y1), color, -1)
            cv2.putText(img_bgr, label, (x1, y1 - 5), 
                       cv2.FONT_HERSHEY_SIMPLEX, 0.6, (255, 255, 255), 2)
        
        output_path = os.path.join(output_folder, f"page_{page_idx:03d}.jpg")
        cv2.imwrite(output_path, img_bgr)
        print(f"  ✓ 頁 {page_idx}: {len(page_dets)} 個物件")
    
    print(f"\n✅ 視覺化完成: {output_folder}")

def generate_excel_report(all_detections, page_counts, output_path):
    """生成 Excel 報告"""
    print(f"\n生成 Excel 報告...")
    
    df = pd.DataFrame(all_detections)
    df = df.sort_values(['page', 'stage', 'class', 'confidence'], 
                        ascending=[True, False, True, False])
    
    with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
        df.to_excel(writer, sheet_name='所有偵測', index=False)
        
        # 階段統計
        stage_summary = df.groupby(['stage', 'class']).size().reset_index(name='數量')
        stage_summary.to_excel(writer, sheet_name='階段統計', index=False)
        
        # 每頁統計
        page_summary = []
        for page, counts in page_counts.items():
            for cls, count in counts.items():
                page_summary.append({'頁碼': page, '類別': cls, '數量': count})
        pd.DataFrame(page_summary).to_excel(writer, sheet_name='每頁統計', index=False)
        
        # 總計
        total_summary = df.groupby('class').size().reset_index(name='總數')
        total_summary.to_excel(writer, sheet_name='總計', index=False)
    
    print(f"✅ 報告已儲存: {output_path}")

print("✅ 視覺化函數定義完成")

In [ ]:
# 生成視覺化和報告
if all_detections:
    visualize_multi_stage_detections(
        pdf_path=PDF_INPUT,
        all_detections=all_detections,
        output_folder=VIS_OUTPUT,
        dpi=INFERENCE_DPI
    )
    
    generate_excel_report(
        all_detections=all_detections,
        page_counts=page_counts,
        output_path=EXCEL_OUTPUT
    )
else:
    print("⚠️ 沒有偵測到任何物件")

## 8️⃣ 診斷工具

In [ ]:
# 分析標註尺寸分佈
print("分析物件尺寸分佈...\n")

label_files = list(Path(AUG_TRAIN_LABELS).glob("*.txt"))
bbox_areas = []
class_areas = {}

for label_file in label_files:
    with open(label_file, 'r') as f:
        for line in f:
            parts = line.strip().split()
            if len(parts) >= 5:
                cls = int(parts[0])
                width = float(parts[3])
                height = float(parts[4])
                area = width * height
                
                bbox_areas.append(area)
                if cls not in class_areas:
                    class_areas[cls] = []
                class_areas[cls].append(area)

if bbox_areas:
    bbox_areas = np.array(bbox_areas)
    print(f"物件尺寸統計 (歸一化面積):")
    print(f"  最小: {bbox_areas.min():.6f}")
    print(f"  平均: {bbox_areas.mean():.6f}")
    print(f"  中位數: {np.median(bbox_areas):.6f}")
    print(f"  最大: {bbox_areas.max():.6f}")
    
    large_count = (bbox_areas >= LARGE_OBJECT_THRESHOLD).sum()
    small_count = (bbox_areas < LARGE_OBJECT_THRESHOLD).sum()
    
    print(f"\n物件分類:")
    print(f"  大物件 (≥{LARGE_OBJECT_THRESHOLD}): {large_count} ({large_count/len(bbox_areas)*100:.1f}%)")
    print(f"  小物件 (<{LARGE_OBJECT_THRESHOLD}): {small_count} ({small_count/len(bbox_areas)*100:.1f}%)")
    
    print(f"\n各類別平均尺寸:")
    for cls, areas in class_areas.items():
        avg_area = np.mean(areas)
        obj_type = "大物件" if avg_area >= LARGE_OBJECT_THRESHOLD else "小物件"
        print(f"  類別 {cls}: {avg_area:.6f} ({obj_type})")